In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Dropout
import keras
import keras_tuner
import keras.utils as image
from keras import layers
from keras import ops
from keras import callbacks
from keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler
from PIL import Image

## Importing Data from csv files


In [6]:
images = pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images.txt", sep=r'\s+', names=['image_id', 'image_name'], engine='python')
train_test_split = pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/train_test_split.txt", sep=r'\s+', names=['image_id', 'is_training_image'], engine='python')
classes =pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/classes.txt", sep=r'\s+', names=['class_id', 'class_name'], engine='python')
image_class_labels =pd.read_csv("/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/image_class_labels.txt", sep=r'\s+', names=['image_id', 'class_id'], engine='python')

## Preprocessing

In [7]:
# Merge dfs based on column names so we have one df with all the necessary info contained per each row
image_data = pd.merge(images,train_test_split, on='image_id')
image_data = pd.merge(image_data,image_class_labels, on='image_id')
image_data = pd.merge(image_data,classes, on='class_id')

In [8]:
# Split training and testing image data
training_image_data = image_data[image_data['is_training_image']==1]
testing_image_data = image_data[image_data['is_training_image']==0]

# Shuffle training data
training_image_data = training_image_data.sample(frac=1)

# Initiate empty lists for training and testing images
training_images = []
testing_images = []

# Add training and testing images to corresponding lists
for i in (training_image_data['image_name'].values):
    training_images.append(image.load_img('/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images/{}'.format(i), target_size=(299, 299)))

for i in (testing_image_data['image_name'].values):
    testing_images.append(image.load_img('/Users/sofie/Desktop/Projects/Classification of Birds/CUB_200_2011/images/{}'.format(i), target_size=(299, 299)))

# Extract class labels for training and testing images
training_class_label = np.array(training_image_data['class_id'].values)
testing_class_label = np.array(testing_image_data['class_id'].values)

In [9]:
# Convert list of images to NumPy array
training_images = np.array(training_images)
testing_images = np.array(testing_images)

# Apply preprocessing
preprocessed_training_images = preprocess_input(training_images)
preprocessed_testing_images = preprocess_input(testing_images)

We begin training by keeping the existing layers frozen, allowing only the newly added classification head to learn. This ensures that the output layer is properly trained before fine-tuning the deeper layers. Once the classifier stabilizes, we gradually unfreeze and fine-tune the earlier layers, optimizing them in a controlled manner. This approach prevents instability in feature extraction and allows each layer to adjust effectively, improving overall model performance.

In [11]:
# Load pre-trained Xception model (without top layers)
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)

# Add new layers on top
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
dense = keras.layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(l2=1e-4))(avg)  # L2 regularization
output = keras.layers.Dense(201, activation="softmax")(dense)  # 201 classes

# Create model
model = keras.Model(inputs=base_model.input, outputs=output)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Define optimizer
optimizer = keras.optimizers.Adam()  # Use Adam with a fixed learning rate

# Compile model
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Set early stopping
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=3, restore_best_weights=True)

# Define validation data split at 30%
split_idx = int(len(preprocessed_training_images) * 0.7)
X_train, X_val = preprocessed_training_images[:split_idx], preprocessed_training_images[split_idx:]
y_train, y_val = training_class_label[:split_idx], training_class_label[split_idx:]

y_train = np.array(y_train).astype(np.int32)
y_val = np.array(y_val).astype(np.int32)

# Train model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32, callbacks=[earlystopping])

# ✅ Save the trained frozen model
model.save("best_frozen_model.h5")

Epoch 1/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.0757 - loss: 4.8448 - val_accuracy: 0.2729 - val_loss: 3.0836
Epoch 2/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 248s 2s/step - accuracy: 0.3759 - loss: 2.6167 - val_accuracy: 0.4352 - val_loss: 2.2756
Epoch 3/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 247s 2s/step - accuracy: 0.5469 - loss: 1.8526 - val_accuracy: 0.5153 - val_loss: 1.9331
Epoch 4/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 250s 2s/step - accuracy: 0.6186 - loss: 1.5177 - val_accuracy: 0.5403 - val_loss: 1.7983
Epoch 5/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 256s 2s/step - accuracy: 0.7014 - loss: 1.2462 - val_accuracy: 0.5420 - val_loss: 1.8012
Epoch 6/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 260s 2s/step - accuracy: 0.7235 - loss: 1.0922 - val_accuracy: 0.5564 - val_loss: 1.7330
Epoch 7/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 250s 2s/step - accuracy: 0.7705 - loss: 0.9539 - val_accuracy: 0.5653 - val_loss: 1.6998
Epoch 8/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 247s 2s/step - accuracy: 0.7921 - loss: 0.9157 - val_accu

In [12]:
# Load the best frozen model
model = keras.models.load_model("best_frozen_model.h5")

# Evaluate on the test set
test_loss, test_acc = model.evaluate(preprocessed_testing_images, testing_class_label)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

182/182 ━━━━━━━━━━━━━━━━━━━━ 239s 1s/step - accuracy: 0.6375 - loss: 1.4508
Test Loss: 1.6009
Test Accuracy: 0.6036


In [32]:
# Load the best frozen model again before fine-tuning
model = keras.models.load_model("best_frozen_model.h5")

# Unfreeze the last 8 layers and add L2 regularization
for layer in model.layers[-8:]:
    layer.trainable = True
    if hasattr(layer, "kernel_regularizer"):
        layer.kernel_regularizer = regularizers.l2(5e-4)  # Apply L2 regularization

# Modify last Dense layer to include Dropout
x = model.layers[-2].output  # Get last Dense layer
dropout = Dropout(0.3)(x)  # Apply dropout before final classification
output = model.layers[-1](dropout)  # Keep the original output layer


# Set early stopping
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights=True)

# Rebuild model with dropout
model = keras.Model(inputs=model.input, outputs=output)

# Recompile with a lower learning rate
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Fine-tune the model
history_fine_tune = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=32, callbacks=[earlystopping])

# Save the fine-tuned model
model.save("best_8layers_unfrozen_model.h5")

Epoch 1/5
 63/132 ━━━━━━━━━━━━━━━━━━━━ 1:41 1s/step - accuracy: 0.7344 - loss: 1.0879

KeyboardInterrupt: 

In [21]:
print(history_fine_tune.history.keys())  # See tracked metrics
print(history_fine_tune.history['val_loss'])  # Print validation loss for all epochs


dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])
[1.5351085662841797, 1.5267738103866577, 1.514609456062317]
